In [2]:
#Import the package to your work environment
import fingertips_py as ftp
import pandas as pd
import altair as alt

In [3]:
# List of variables (Indicator IDs)
indicators = {
    "Referrals (Rate)": 90747,
    "Entering Treatment (Rate)": 90798,
    "Completion Rates for IAPT Treatment": 90748
}

# Fetch data for each indicator and combine into a single DataFrame
data = pd.concat(
    [ftp.get_data_for_indicator_at_all_available_geographies(indicator) for indicator in indicators.values()],
    ignore_index=True
)


In [11]:
data.head(2)

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to percentiles,Time period Sortable,New data,Compared to goal,Time period range
0,90747,"IAPT referrals: rate (quarterly) per 100,000 p...",NaN,NaN,E92000001,England,England,Persons,18+ yrs,NaN,...,259042.0,42070419.0,NaN,NaN,Not compared,Not compared,20130200,NaN,NaN,3m
1,90747,"IAPT referrals: rate (quarterly) per 100,000 p...",E92000001,England,E92000001,England,England,Persons,18+ yrs,NaN,...,259042.0,42070419.0,NaN,NaN,Not compared,Not compared,20130200,NaN,NaN,3m


In [5]:
# Filter data for England only
england_data = data[data["Area Name"] == "England"].copy()

# Extract year from the 'Time period Sortable' column (e.g., 20130200 -> 2013)
england_data["Year"] = england_data["Time period Sortable"].astype(str).str[:4]

# Aggregate by year: calculate the mean referral rate (Value)
england_yearly = england_data.groupby("Year", as_index=False)["Value"].mean()

In [6]:
# Display unique values in the "Indicator Name" column
uniques = england_data["Indicator Name"].unique()

# Print the unique values
print(uniques)

['IAPT referrals: rate (quarterly) per 100,000 population'
 'Entering IAPT treatment: rate (quarterly) per 100,000 population'
 'Completion of IAPT treatment: rate (quarterly) per 100,000 population']


In [11]:
# Filter to include only quarterly data
england_quarterly = england_data[england_data["Time period"].str.contains("Q", na=False)].copy()

# Extract the starting year from the "Time period" column (e.g., "2013/14 Q1" → "2013")
england_quarterly["Year"] = england_quarterly["Time period"].str.extract(r'(\d{4})')

# Group by Year and calculate the average referral rate
england_yearly_avg = england_quarterly.groupby("Year", as_index=False)["Value"].mean()

# Create a bar chart
yearly_chart = alt.Chart(england_yearly_avg).mark_bar().encode(
    x=alt.X("Year:N", title="Year"),  # Treat year as a nominal (categorical) variable
    y=alt.Y("Value:Q", title="Average IAPT Referrals Rate (per 100,000)"),
    tooltip=[
        alt.Tooltip("Year:N", title="Year"),
        alt.Tooltip("Value:Q", title="Referral Rate")
    ]
).properties(
    width=800,
    height=400,
    title="Yearly Average IAPT Referrals Rate in England (per 100,000 population)"
)

yearly_chart



alt.Chart(...)

In [12]:
# Define a professional colorblind-friendly palette
color_palette = {
    "IAPT referrals: rate (quarterly) per 100,000 population": "#084081",  
    "Entering IAPT treatment: rate (quarterly) per 100,000 population": "#4eb3d3", 
    "Completion of IAPT treatment: rate (quarterly) per 100,000 population": "#ccebc5"  
}

# Ensure grouping includes both Year and Indicator Name
england_yearly_avg = england_quarterly.groupby(["Year", "Indicator Name"], as_index=False)["Value"].mean()

# Create the improved stacked bar chart
stacked_chart = alt.Chart(england_yearly_avg).mark_bar().encode(
    x=alt.X("Year:N", title="Year"),
    y=alt.Y("Value:Q", title="Average Rate (per 100,000 population)", axis=alt.Axis(grid=True)),
    color=alt.Color(
        "Indicator Name:N",
        scale=alt.Scale(domain=list(color_palette.keys()), range=list(color_palette.values())),
        title="Indicator"
    ),
    tooltip=[
        alt.Tooltip("Year:N", title="Year"),
        alt.Tooltip("Indicator Name:N", title="Indicator"),
        alt.Tooltip("Value:Q", title="Rate", format=",.2f")
    ]
).properties(
    width=800,
    height=400,
    title=alt.TitleParams(
        text="Yearly Average Rates by IAPT Indicator",
        subtitle="Source: Fingertips | Rate per 100,000 Population",
        font="Arial",
        fontSize=16,
        subtitleFont="Arial",
        subtitleFontSize=12,
        anchor="middle",
        offset=20
    )
).configure_legend(
    titleFont="Arial",
    titleFontSize=14,
    labelFont="Arial",
    labelFontSize=12,
    orient="top-right",  # Move legend to the top-right
    padding=10  # Add padding between chart and legend
).configure_axis(
    labelFont="Arial",
    labelFontSize=12,
    titleFont="Arial",
    titleFontSize=14
).configure_title(
    fontSize=18,
    font="Arial",
    anchor="middle"
).configure_view(
    strokeWidth=0  # Remove chart border
)

stacked_chart



alt.Chart(...)

In [13]:
# Create the improved stacked bar chart with horizontal x-axis labels
stacked_chart = alt.Chart(england_yearly_avg).mark_bar().encode(
    x=alt.X("Year:N", title="Year", axis=alt.Axis(labelAngle=0)),  # Horizontal x-axis labels
    y=alt.Y("Value:Q", title="Average Rate (per 100,000 population)", axis=alt.Axis(grid=True)),
    color=alt.Color(
        "Indicator Name:N",
        scale=alt.Scale(domain=list(color_palette.keys()), range=list(color_palette.values())),
        title="Indicator"
    ),
    tooltip=[
        alt.Tooltip("Year:N", title="Year"),
        alt.Tooltip("Indicator Name:N", title="Indicator"),
        alt.Tooltip("Value:Q", title="Rate", format=",.2f")
    ]
).properties(
    width=800,
    height=400,
    title=alt.TitleParams(
        text="Yearly Average Rates by IAPT Indicator",
        subtitle="Source: Fingertips | Rate per 100,000 Population",
        font="Arial",
        fontSize=16,
        subtitleFont="Arial",
        subtitleFontSize=12,
        anchor="middle",
        offset=20
    )
).configure_legend(
    titleFont="Arial",
    titleFontSize=14,
    labelFont="Arial",
    labelFontSize=12,
    orient="top-right",  # Move legend to the top-right
    padding=10  # Add padding between chart and legend
).configure_axis(
    labelFont="Arial",
    labelFontSize=12,
    titleFont="Arial",
    titleFontSize=14
).configure_title(
    fontSize=18,
    font="Arial",
    anchor="middle"
).configure_view(
    strokeWidth=0  # Remove chart border
)

stacked_chart


alt.Chart(...)

In [30]:
# Create the improved stacked bar chart with adjusted size and padding for the legend
stacked_chart = alt.Chart(england_yearly_avg).mark_bar().encode(
    x=alt.X("Year:N", title="Year", axis=alt.Axis(labelAngle=0)),  # Horizontal x-axis labels
    y=alt.Y("Value:Q", title="Average Rate (per 100,000 population)", axis=alt.Axis(grid=True)),
    color=alt.Color(
        "Indicator Name:N",
        scale=alt.Scale(domain=list(color_palette.keys()), range=list(color_palette.values())),
        title="Indicator"
    ),
    tooltip=[
        alt.Tooltip("Year:N", title="Year"),
        alt.Tooltip("Indicator Name:N", title="Indicator"),
        alt.Tooltip("Value:Q", title="Rate", format=",.2f")
    ]
).properties(
    width=900,  # Increase chart width
    height=450,  # Increase chart height for better readability
    title=alt.TitleParams(
        text="Yearly Average Rates by IAPT Indicator",
        subtitle="Source: Fingertips | Rate per 100,000 Population",
        font="Arial",
        fontSize=16,
        subtitleFont="Arial",
        subtitleFontSize=12,
        anchor="middle",
        offset=20
    )
).configure_legend(
    titleFont="Arial",
    titleFontSize=14,
    labelFont="Arial",
    labelFontSize=11,
    orient="top-right",  # Keep legend on the top-right
    padding=25  # Add more padding for the legend
).configure_axis(
    labelFont="Arial",
    labelFontSize=12,
    titleFont="Arial",
    titleFontSize=14
).configure_title(
    fontSize=18,
    font="Arial",
    anchor="middle"
).configure_view(
    strokeWidth=0  # Remove chart border
)

stacked_chart


alt.Chart(...)

In [23]:
import os

# Define the file path and name
output_path = r"C:\Bruno\LSE\PP434 - Automated Data Visualisation\Week 1\Code Examples"
file_name = "chart7_IAPT_referalsentries.json"

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Save the chart as JSON
chart_path = os.path.join(output_path, file_name)
stacked_chart.save(chart_path)

print(f"Chart saved successfully to {chart_path}")

Chart saved successfully to C:\Bruno\LSE\PP434 - Automated Data Visualisation\Week 1\Code Examples\chart7_IAPT_referalsentries.json
